In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime
import os

import io

import matplotlib.pyplot as plt
import seaborn as sns


#### Interactivity 
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, Layout
from IPython.display import display
from IPython.display import display, clear_output


style = {'description_width': 'initial'}

REFs = widgets.FileUpload(accept='*.csv', multiple=False, description = 'Referrals CSV', 
                          layout=Layout(width = '250px'), style=style)
display(REFs)


CINs = widgets.FileUpload(accept='*.csv', multiple=False, description = 'CIN plan CSV', 
                          layout=Layout(width = '250px'), style=style)
display(CINs)

CPs = widgets.FileUpload(accept='*.csv', multiple=False, description = 'CPs plans CSV', 
                          layout=Layout(width = '250px'), style=style)
display(CPs)

CICs = widgets.FileUpload(accept='*.csv', multiple=False, description = 'CLA status CSV', 
                          layout=Layout(width = '250px'), style=style)
display(CICs)


IDColumns = []
def id_column(RefID, CINID, CICID, CPID):
    IDColumns.append(RefID)
    IDColumns.append(CINID)
    IDColumns.append(CICID)
    IDColumns.append(CPID)
    return IDColumns

interact_manual.options(manual_name = 'Submit headers')(id_column, 
                RefID = widgets.Text(value = 'LEGACY_ID', placeholder = 'LEGACY_ID', description = 'Column header with child ID: Ref table',
                                    layout=Layout(width = '500px'), style=style),
                CINID = widgets.Text(value = 'PersonId', placeholder = 'PersonId', description = 'Column header with child ID: CIN table',
                                    layout=Layout(width = '500px'), style=style),
                CICID = widgets.Text(value = 'ChildUniqueID', placeholder = 'ChildUniqueID', description = 'Column header with child ID: CLA table',
                                    layout=Layout(width = '500px'), style=style),
                CPID = widgets.Text(value = 'LEGACY_ID2', placeholder = 'LEGACY_ID2', description = 'Column header with child ID: CP table',
                                    layout=Layout(width = '500px'), style=style)
               )


DateColumns = []
def date_column(Refd, CINd, CICd, CPd):
    DateColumns.append(Refd)
    DateColumns.append(CINd)
    DateColumns.append(CICd)
    DateColumns.append(CPd)
    return DateColumns


interact_manual.options(manual_name = 'Submit headers')(date_column, 
                Refd = widgets.Text(value = 'REFRL_START_DTTM', placeholder = 'REFRL_START_DTTM', description = 'Column header with child ID: Ref table',
                                    layout=Layout(width = '500px'), style=style),
                CINd = widgets.Text(value = 'CINStartDate', placeholder = 'CINStartDate', description = 'Column header with child ID: CIN table',
                                    layout=Layout(width = '500px'), style=style),
                CICd = widgets.Text(value = 'LegalStatusStart', placeholder = 'LegalStatusStart', description = 'Column header with child ID: CLA table',
                                    layout=Layout(width = '500px'), style=style),
                CPd = widgets.Text(value = 'START_DTTM2', placeholder = 'START_DTTM2', description = 'Column header with child ID: CP table',
                                    layout=Layout(width = '500px'), style=style)
                        )


In [ ]:

RefMatch = IDColumns[0]
CINMatch = IDColumns[1]
CICMatch = IDColumns[2]
CPMatch = IDColumns[3]


REFdate = DateColumns[0]
CINdate = DateColumns[1]
CLAdate = DateColumns[2]
CPdate = DateColumns[3]

In [ ]:
input_file = list(REFs.value.values())[0]
content = input_file['content']
content = io.StringIO(content.decode('utf-8'))
REF = pd.read_csv(content)

input_file2 = list(CINs.value.values())[0]
content2 = input_file2['content']
content2 = io.StringIO(content2.decode('utf-8'))
CIN = pd.read_csv(content2, skiprows = 3)

input_file3 = list(CICs.value.values())[0]
content3 = input_file3['content']
content3 = io.StringIO(content3.decode('utf-8'))
CIC = pd.read_csv(content3)

input_file4 = list(CPs.value.values())[0]
content4 = input_file4['content']
content4 = io.StringIO(content4.decode('utf-8'))
CP = pd.read_csv(content4)


In [ ]:
def id_merger(Table1, Table2, Match1, Match2, NewName):
    '''This function is used to merge tables together based on unique child ID using the table names and column names'''
    NewName =pd.merge(Table1, Table2, how = 'inner', left_on = Match1, right_on = Match2)
    return NewName

REFCIN = ()
REFCIN = id_merger(REF, CIN, RefMatch, CINMatch, REFCIN)

REFCP = ()
REFCP = id_merger(REF, CP, RefMatch, CPMatch, REFCP)

REFCIC = ()
REFCIC = id_merger(REF, CIC, RefMatch, CICMatch, REFCIC)

CINCP = ()   
CINCP = id_merger(CIN, CP, CINMatch, CPMatch, CINCP)

CINCIC = ()
CINCIC = id_merger(CIN, CIC, CINMatch, CICMatch, CINCIC)

CPCIC = ()
CPCIC = id_merger(CP, CIC, CPMatch, CICMatch, CPCIC)

In [ ]:
REFCPimportant = () 
REFCICimportant = () 
REFCINimportant = ()
CINCLAimportant = ()
CINCPimportant = ()
CPCICimportant = ()

def StageWait(StartColumn1, StartColumn2, MergedTable, NewTableName, PrintStats):
    NewTableName=MergedTable.loc[:,[StartColumn1,StartColumn2]]
    
    NewTableName[StartColumn1] = pd.to_datetime(NewTableName[StartColumn1])
    NewTableName[StartColumn2] = pd.to_datetime(NewTableName[StartColumn2])
    
    NewTableName['Time Gap'] = NewTableName[StartColumn2] - NewTableName[StartColumn1]
    
    NewTableName = NewTableName[NewTableName['Time Gap'] > datetime.timedelta(days=0)]
    
    NewTableName['Time Gap float'] = NewTableName['Time Gap'].astype('timedelta64[h]')
    NewTableName['Time Gap float'] =  NewTableName['Time Gap float'].astype(float)/24
    
    return NewTableName
    #print(NewTableName.head())

StatsOutput='Yes'
#REFCIC
REFCICimportant = StageWait(REFdate, CLAdate, REFCIC, REFCICimportant, StatsOutput)
#REFCP
REFCPimportant = StageWait(REFdate, CPdate, REFCP, REFCPimportant, StatsOutput)
#REFCIN
REFCINimportant = StageWait(REFdate, CINdate, REFCIN, REFCINimportant, StatsOutput)
#CINCIC
CINCICimportant = StageWait(CINdate, CLAdate, CINCIC, CINCLAimportant, StatsOutput)
#CINCP
CINCPimportant = StageWait(CINdate, CPdate, CINCP, CINCPimportant, StatsOutput)
#CPCIC
CPCICimportant = StageWait(CPdate, CLAdate, CPCIC, CINCPimportant, StatsOutput)


In [ ]:
def CasesByYearSort():
    '''This function creates a new dataframe with cases and average wait times for cases starting in each year.'''
    CINCICimportant['year']=CINCICimportant['CINStartDate'].dt.year

    WaitByYear = CINCICimportant.groupby(CINCICimportant['year'])['Time Gap float'].mean()

    CasesByYear = CINCICimportant.value_counts('year').rename_axis('year').reset_index(name='Cases startign that year')

    WaitCasesByYear = pd.merge(WaitByYear, CasesByYear, on='year') 
    WaitCasesByYear.columns = ['year', 'Average wait', 'Cases starting that year']
    return WaitCasesByYear
WaitCasesByYear = CasesByYearSort()

In [ ]:
WBSYdf = ()
WBSYCINCIC = ()
def wait_by_start_year(years):
    
    WBSYdf = WaitCasesByYear[(WaitCasesByYear['year'] >=years[0]) & (WaitCasesByYear['year'] <=years[1])]
    WBSYCINCIC=CINCICimportant[(CINCICimportant['year'] >=years[0]) & (CINCICimportant['year'] <=years[1])]
    
    #Sets the colour palette to coolwarm_r and its length to the length of the dataframe
    pal = sns.color_palette("coolwarm_r", len(WBSYdf))
    #.aggsort 1 returns the indices int he order that would sort the column, the second provides the indicies
    # that would sort the first set of indices.
    rank = WBSYdf['Average wait'].argsort().argsort()   
    
    #Sets seaborn palette style.
    sns.set_style('darkgrid', {"axes.facecolor": ".9"})

    #Initialises the figure with two sets of axes.
    fig, axes = plt.subplots(2, 1, figsize=(20,10), sharex=True)
    
    #Title according to fucntion inputs.
    plt.suptitle('MainTitle')

    # Makes a barplot at axes index 0 with WaitCasesByYear as the data, the year column as x and Average wait colum
    # as y. Then sets the palette according to the pal and rank variables above.  
    sns.barplot(ax=axes[0], data=WBSYdf, x='year', y='Average wait', palette=np.array(pal[::-1])[rank])
    
    #Puts a horizontal like at the mean value for wait times, like the vertical ones above.
    axes[0].axhline(WBSYCINCIC['Time Gap float'].mean(), ls='--', color = 'navy')
    #Adds descriptor text to the mean bar.
    axes[0].text(0,-200, 'Average wait for all years: ----', color = 'navy', fontsize=20)
    
    #Sets axes titles/
    axes[0].set_title('WCBY')
    axes[0].set_xlabel('')
    axes[0].set_ylabel('Average wait (Days)')

    i=-1 #necessary to start at 0
    for p in axes[0].patches:
        i=i+1
            # Accesses WaitCasesByYear by index location mathcing i and returns percentage as string to be annotate.
            # Gets height and location of top of bar for annotation and places va ha center, rotates.
        axes[0].annotate(str(WaitCasesByYear['Cases starting that year'].iloc[i]), (p.get_x() + p.get_width() / 2., p.get_height()),
                 ha='center', va='center', fontsize=15, color='black', xytext=(0, 5),
                 textcoords='offset points')

    
    
    #Plots the second histogram using rules from the first.
    #The palette is left the same as the averages are the same.
    sns.boxplot(ax=axes[1], data=WBSYCINCIC, x='year', y='Time Gap float',  palette=np.array(pal[::-1])[rank])
    axes[1].axhline(WBSYCINCIC['Time Gap float'].mean(), ls='--', color = 'navy')
    axes[1].set_title('CINCIC')
    axes[1].set_xlabel('Year')
    axes[1].set_ylabel('Wait (Days)')
    
    plt.show()

    
interact(wait_by_start_year, years = widgets.IntRangeSlider(
    value=[2003, 2021],
    min=2003,
    max=2021,
    step=1,
    description='Year Range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'))

In [ ]:
GraphDict = {'Ref to CIN' : REFCINimportant, 
            'Ref to CP' : REFCPimportant, 
            'Ref to CIC' : REFCICimportant, 
            'CIN to CP' : CINCPimportant,
            'CIN to CIC' : CINCICimportant, 
            'CP to CIC' : CPCICimportant}

def wait_hist_plotter2(Data, Title):
    sns.set_style('darkgrid', {"axes.facecolor": ".9"})
    sns.set_palette("flare")
    fig = sns.histplot(data=Data, x='Time Gap float')
    fig.set_title(Title)
    fig.set_xlabel('Wait time (Days)')
    fig.set_ylabel('Number of Children')
    fig.axvline(Data['Time Gap float'].mean(), ls='--', color = 'navy')

    plt.show()

def graph_selector2(graphs):
    
    for i in range(len(graphs)):
        title = graphs[i]
        data = GraphDict[graphs[i]]
        wait_hist_plotter2(data, title)
        
interact_manual(graph_selector2, graphs=widgets.SelectMultiple(
    options=GraphDict.keys()
    ))
